In [219]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sys
import os

import personalized_neural_net
print(os.getcwd())
%pylab inline

reload(neural_net)
neural_net.reload_files()

dropbox_path = '/Users/nd/projects/pml-neuralnet-hw/PML-DNN-hw1/'
filename = 'parkinsons_supervised.csv'

/Users/nd/projects/pml-neuralnet-hw/PML-DNN-hw1/neural_net
Populating the interactive namespace from numpy and matplotlib


In [220]:
layer_sizes = [128,64,32]
learning_rate = 0.001
weight_penalty = 0.01
dropout_prob = 0.9
batch_size = 25
clip_gradients = True
results_dict = {}

for i in range(0, 5):
    net = personalized_neural_net.NeuralNetwork(dropbox_path + filename, 'homework', 
                               layer_sizes, batch_size, learning_rate, dropout_prob, 
                               weight_penalty, clip_gradients, model_type='regression')

    net.train(num_steps=50000, output_every_nth=10000)
    results_dict[(str(layer_sizes[0]) + "x" + str(layer_sizes[1]), learning_rate, weight_penalty, dropout_prob, i)] = net.test_on_validation()

skipping normalization for subject_num
Original data length was 5875
After dropping rows with nan in any label column, length is 5875
3581 rows in training data
val!
(array([[  1.        ,   0.81216377,  -0.68470936, ...,   0.23514099,
         -1.20083516,   1.55027781],
       [ 38.        ,   0.2483973 ,  -0.68470936, ...,  -0.15707945,
         -0.54076733,  -0.81125546],
       [  8.        ,   0.92491706,   1.46047368, ...,  -0.29225831,
          0.21677056,  -0.34420179],
       ..., 
       [  8.        ,   0.92491706,   1.46047368, ...,  -1.99531016,
         -1.14809162,  -0.52761651],
       [ 41.        ,   0.36115059,   1.46047368, ...,   0.90306578,
          0.7741643 ,   0.38374258],
       [ 10.        ,  -0.76638234,  -0.68470936, ...,   0.51538493,
         -1.60998439,  -0.93719511]]), array([[ 36.443,  47.257],
       [ 18.256,  23.513],
       [ 18.702,  24.702],
       ..., 
       [ 20.565,  26.565],
       [ 35.579,  43.579],
       [ 12.   ,  19.   ]]))
1167 

In [20]:
#find good hyperparameters for the base set

layer_size_cands = [[64, 32], [128, 64, 32], [32, 32, 32], [32, 16, 8]]
learning_rates = [0.01, .1]
weight_penalties = [0.01, .05]
dropouts = [0.9, .8]




def search_for_HPs():
    rmse_results = []
    dict = {}
    import time
    start = time.time()
    index = 0
    for learning_rate in learning_rates:
        for weight_penalty in weight_penalties:
            for dropout_prob in dropouts:
                for i in range(0,5):
                    clip_gradients = True
                    print('this is iteration ' + str(i) + ' !!!!!')
                    net = neural_net.NeuralNetwork(dropbox_path + filename, 'homework', 
                                               layer_size_cands[index], batch_size, learning_rate, dropout_prob, 
                                               weight_penalty, clip_gradients, model_type='regression')
                    net.train(num_steps=50000, output_every_nth=10000)
                    dict[(str(layer_size_cands[index][0]) + "x" + str(layer_size_cands[index][1]), learning_rate, weight_penalty, dropout_prob, i)] = net.test_on_validation()

    end = time.time()
    print end-start
    net.plot_training_progress()


### debugging neural network using session.run

In [19]:
print net.session.run([net.tf_X,net.tf_Y, net.logits], feed_dict=feed_dict)
#print "X", np.shape(tf_X), "Y", np.shape(tf_Y), "logits", np.shape(logits)

[array([[ 0.35033551, -1.78116202, -0.68470937, -1.40052152, -0.30320659,
        -0.1525038 , -0.25575474, -0.20323569, -0.25678062,  0.96460992,
         0.52912575,  0.97140121,  1.18462873,  0.82155502,  0.97114718,
        -0.20225847, -0.40879247,  1.4391408 ,  0.14954013, -0.16320474],
       [-0.62218499, -0.76638234,  1.46047366,  0.49315697, -0.65916598,
        -0.88036144, -0.54216307, -0.54065609, -0.54115933, -0.91260463,
        -0.95570928, -0.86152756, -0.82540822, -0.99184608, -0.86126816,
        -0.48749965,  1.05310428, -2.19696879,  0.07246424, -1.14588571],
       [ 0.4313789 , -0.87913561,  1.46047366, -1.33126593, -0.18627307,
        -0.4621276 , -0.04247193, -0.14307547, -0.0434966 ,  0.47931957,
         0.38813251,  0.52619565,  0.54269242,  0.58743757,  0.52594227,
        -0.46723726,  0.82877713, -1.87455714,  1.49147642, -0.31518951],
       [-0.70322841,  0.92491704,  1.46047366, -0.54672247, -0.43217739,
        -0.71460325, -0.29841128, -0.35755974, 

In [18]:
# To see the tensorflow model you've built in tensorboard, run the following
journalist = tf.summary.FileWriter(net.checkpoint_dir, graph=net.session.graph)
journalist.flush()

# Now enter the following command at the command line
# > tensorboard --logdir 'saved_models'
# (you may need to replace 'saved_models' with the path to the actual 
# net.checkpoint_dir you are using)